In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Reshape

In [2]:
autoencoder = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(256, activation='sigmoid'),
    Dense(64, activation='sigmoid'),
    Dense(2, activation='sigmoid'),
    Dense(64, activation='sigmoid'),
    Dense(256, activation='sigmoid'),
    Dense(784, activation='sigmoid'),
    Reshape((28,28))
])

An autoencoder could be viewed as a compression algorithm similar to VAE, although it is not a probabilistic model and it isn't modeling the underlying data distribution.

The autoencoder has a bottleneck architecture. The width of the dense layers is decreasing at first, all the way down to the middle bottleneck layer with just two neurons. The network then starts to widen out again afterward until the final layer which is the same size and shape as the input layer.

When we train an autoencoder we feed it the same data examples for the input as for the output, so this is an unsupervised learning algorithm as we are not using any labels. Instead the model is being trained to reconstruct the data example that it receives as the output of the network. The bottleneck architecture forces the network to learn a compressed representation which captures the main features of the data. It is able afterward to reconstruct the data examples from this compressed representation.

In practice it is useful to separate the encoder and the decoder parts of the network architecture. The bottleneck layer in the network with just 2 neurons is the key for the network to learn to compress the data input. So everything up and including that dense layer is the encoding part of the network. 

In [3]:
encoder = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(256, activation='sigmoid'),
    Dense(64, activation='sigmoid'),
    Dense(2, activation='sigmoid')
])

decoder = Sequential([
    Dense(64, activation='sigmoid', input_shape=(2,)),
    Dense(256, activation='sigmoid'),
    Dense(784, activation='sigmoid'),
    Reshape((28,28))
])

The encoder network takes a batch of data examples as input and returns a batch of latent codes that compress those examples.

The second half of the autoencoder is the decoder network. The decoder network takes a batch of latent codes of length 2 and decompresses them. The output of the network is 28 by 28, the same size as the input for the encoder.

In [ ]:
autoencoder = Model(inputs=encoder.input, outputs=decoder(encoder.output))

autoencoder.compile(loss='mse', optimizer='sgd')
autoencoder.fit(x_train, x_train, epochs=20)

The complete autoencoder network can then be defined as above. The input for the autoencoder is the input tensor to the encoder network and the output is the output tensor of the decoder, after decompressing the latent code output produced by the encoder. The autoencoder can then be compile and trained as normal.

In [ ]:
# x_test: (1, 28, 28)
reconstruction = autoencoder(x_test) # (1, 28, 28)

The first dimension 1 is the batch size. We can run our test instance by the full autoencoder, the output will be a tensor of shape (1,28,28), the same as the input and this will be the reconstruction of the data input by the autoencoder after squeezing the data input through the bottleneck latent code layer of just 2 neurons.

In [ ]:
x_encoded = encoder(x_test) # (1,2)

Also, we can easily compute the latent code representation of the encoder network like in the above. We just need to pass x_test to the encoder and the output will be a tensor of shape (batch_size, 2), because the latent code as been set to have size 2. We can think of this output of being the encoded version of x_test.

In [ ]:
z = tf.random.normal([1, 2])
z_decoded = decoder(z) # (1, 28, 28)

Finally, we can also pick a latent code chosen at random and pass it through the decoder. The output will be the decompressed version of the latent code z, which has shape (1, 28, 28).